In [1]:
import sys
import os
import re
import h5py
import gc
import json
import numpy as np
from modules.module import load_config_file, pair, get_elicitation_time_by_subject, load_frames, load_gaze_and_diameter
from modules.preprocessing import preprocess_gaze_pupil_data

In [2]:
user_label = load_config_file("user_label.json")

config = load_config_file("modules/dataset_config.json")

frame_size = pair(config['periocular_resolution']) if config['periocular_resolution'] else sys.exit("Periocular resolution is not specified in the config file.")

subject_dirs = sorted([
    d for d in os.listdir(config['raw_data_dir'])
    if os.path.isdir(os.path.join(config['raw_data_dir'], d))
])

print(subject_dirs)

{'0a': {'all': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, '0b': {'all': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, '1a': {'0:35': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, '1b': {'1:26': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, '2a': {'0:19': [1, 2, 3, 5, 7, 8, 10, 11, 12, 14, 16, 17], '1:20': [4, 6, 9, 13, 15, 18, 19, 20]}, '2b': {'0:06': [1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19], '0:45': [4, 16], '1:30': [7, 20]}, '3a': {'1:37': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 19, 20], '2:00': [11, 17, 18]}, '3b': {'0:55': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}, '4a': {'0:32': [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20], '1:35': [2, 14]}, '4b': {'0:31': [9], '0:56': [1, 2, 3, 4, 5, 6, 8, 11, 12, 13, 15, 17, 19, 20], '1:23': [7, 10, 14, 16, 18]}, '5a': {'0:38': [1, 2, 3, 4, 5, 6, 7, 8, 9

In [3]:
min_index_counter = 0
max_index_counter = 0
session_amount_counter = 0

all_subjects_index_info = {}

destination_dir = os.path.join(config['destination_dir'], f"window_{config['window_time']}_fps_{config['periocular_sample_rate']}_fsize_{config['periocular_resolution']}_fchannel_{config['periocular_channels']}_gaze_pupil_rate_{config['gaze_pupil_sample_rate']}_overlap_{config['overlap']}")

os.makedirs(destination_dir, exist_ok=True)

for subject in subject_dirs:

    # Get the subject number
    subject_number = int(subject.replace('P', ''))


    session_dirs = sorted([
        d for d in os.listdir(os.path.join(config['raw_data_dir'], subject))
        if os.path.isdir(os.path.join(config['raw_data_dir'], subject, d))
    ])

    subject_h5_file = os.path.join(destination_dir, f"{subject}.h5")

    for session in session_dirs:
        # Get the elicitation time of the session
        elicitation_time = get_elicitation_time_by_subject(user_label, session, subject_number)

        match = re.match(r'(\d+)([a-z])', session.lower())
        if match:
            session_number = int(match.group(1))
            session_type = match.group(2)
        else:
            sys.exit("Error: The provided string does not match the required format (number followed by a lowercase letter).")

        if elicitation_time == "none":
            print(f"Subject {subject} claimed no emotion elicitation for session {session}")
        else:
            # Calculate the elicitation time in seconds
            if elicitation_time == "all":
                elicitation_time_in_seconds = 0
            else:
                minutes, seconds = elicitation_time.split(':')
                elicitation_time_in_seconds = int(minutes) * 60 + int(seconds)

            # Process the periocular recordings
            right_recording_path = os.path.join(config['raw_data_dir'], subject, session, 'periocular', 'eye0.mp4')
            left_recording_path = os.path.join(config['raw_data_dir'], subject, session, 'periocular', 'eye1.mp4')

            inversed_right_frames, right_duration = load_frames(right_recording_path, config['window_time'], elicitation_time_in_seconds, 120, config['periocular_sample_rate'], preprocess='none', resize=pair(config['periocular_resolution']), frame_channels=config['periocular_channels'], overlap=config['overlap'])
            inversed_left_frames, left_duration = load_frames(left_recording_path, config['window_time'], elicitation_time_in_seconds, 120, config['periocular_sample_rate'], preprocess='flip_horizontal', resize=pair(config['periocular_resolution']), frame_channels=config['periocular_channels'], overlap=config['overlap'])

            # # Process the scene recording
            # scene_recording_path = os.path.join(config['raw_data_dir'], subject, session, 'scene', 'world.mp4')

            # inversed_scene_frames, scene_duration = load_frames(scene_recording_path, config['window_time'], elicitation_time_in_seconds, 30, config['scene_sample_rate'], preprocess='none', resize=pair(config['scene_resolution']), frame_channels=config['scene_channels'], overlap=config['overlap'])

            # Process the gaze and pupil recording
            gaze_path = os.path.join(config['raw_data_dir'], subject, session, 'gaze_pupil', 'gaze.csv')
            pupil_path = os.path.join(config['raw_data_dir'], subject, session, 'gaze_pupil', 'pupil.csv')

            gaze, pupil = preprocess_gaze_pupil_data(gaze_path, pupil_path, save_path=None)

            inversed_gaze_pupil = load_gaze_and_diameter(gaze, pupil, config['window_time'], config['gaze_pupil_sample_rate'], min(right_duration, left_duration), config['overlap'])

            # Alginment
            min_length = min(len(inversed_right_frames), len(inversed_left_frames), len(inversed_gaze_pupil))
            inversed_right_frames = inversed_right_frames[:min_length]
            inversed_left_frames = inversed_left_frames[:min_length]
            inversed_gaze_pupil = inversed_gaze_pupil[:min_length]

            # Flip to inverse to the right order
            right_frames = np.flip(inversed_right_frames, axis=0)
            right_frames /= 255.0  # Normalize the pixel values to the range [0, 1]
            left_frames = np.flip(inversed_left_frames, axis=0)
            left_frames /= 255.0  # Normalize the pixel values to the range [0, 1]
            gaze_pupil = np.flip(inversed_gaze_pupil, axis=0)

            label_dtype = np.dtype([
                ('subject', 'i8'),
                ('label', 'i8'),
                ('session_label', 'S1')
            ])

            label = np.array([(subject_number, session_number, session_type)] * min_length, dtype=label_dtype)

            with h5py.File(subject_h5_file, 'a') as h5f:
                # Use chunking and compression for datasets
                chunk_size = (1,) + right_frames.shape[1:]  # Chunk size of 1 batch

                if 'labels' in h5f:
                    h5f['labels'].resize((h5f['labels'].shape[0] + min_length,))
                    h5f['labels'][-min_length:] = label
                else:
                    h5f.create_dataset('labels', data=label, maxshape=(None,), dtype=label_dtype,
                                    chunks=True, compression="gzip", compression_opts=4)

                if 'gaze_pupil' in h5f:
                    h5f['gaze_pupil'].resize((h5f['gaze_pupil'].shape[0] + min_length,) + gaze_pupil.shape[1:])
                    h5f['gaze_pupil'][-min_length:] = gaze_pupil
                else:
                    h5f.create_dataset('gaze_pupil', data=gaze_pupil, maxshape=(None,) + gaze_pupil.shape[1:],
                                    dtype=np.float32, chunks=True, compression="gzip", compression_opts=4)

                if 'eye0' in h5f:
                    h5f['eye0'].resize((h5f['eye0'].shape[0] + min_length,) + right_frames.shape[1:])
                    h5f['eye0'][-min_length:] = right_frames
                else:
                    h5f.create_dataset('eye0', data=right_frames, maxshape=(None,) + right_frames.shape[1:],
                                    dtype=np.float32, chunks=chunk_size, compression="gzip", compression_opts=4)

                if 'eye1' in h5f:
                    h5f['eye1'].resize((h5f['eye1'].shape[0] + min_length,) + left_frames.shape[1:])
                    h5f['eye1'][-min_length:] = left_frames
                else:
                    h5f.create_dataset('eye1', data=left_frames, maxshape=(None,) + left_frames.shape[1:],
                                    dtype=np.float32, chunks=chunk_size, compression="gzip", compression_opts=4)

            # Clean up
            del right_frames, left_frames, gaze_pupil, label
            gc.collect()

            max_index_counter += min_length

            session_label = f"{session}_amount"
            if subject not in all_subjects_index_info:
                all_subjects_index_info[subject] = {}

            all_subjects_index_info[subject][session_label] = max_index_counter - session_amount_counter
            session_amount_counter = max_index_counter

    all_subjects_index_info[subject]["min_index"] = min_index_counter

    all_subjects_index_info[subject]["max_index"] = max_index_counter - 1
    
    # Update the index_counter for the next subject
    min_index_counter = max_index_counter

frames_array shape: (43, 10, 224, 224, 1)
frames_array shape: (43, 10, 224, 224, 1)
frames_array shape: (31, 10, 224, 224, 1)
frames_array shape: (31, 10, 224, 224, 1)
frames_array shape: (5, 10, 224, 224, 1)
frames_array shape: (5, 10, 224, 224, 1)
frames_array shape: (10, 10, 224, 224, 1)
frames_array shape: (10, 10, 224, 224, 1)
frames_array shape: (133, 10, 224, 224, 1)
frames_array shape: (133, 10, 224, 224, 1)
frames_array shape: (108, 10, 224, 224, 1)
frames_array shape: (108, 10, 224, 224, 1)
frames_array shape: (46, 10, 224, 224, 1)
frames_array shape: (47, 10, 224, 224, 1)
frames_array shape: (53, 10, 224, 224, 1)
frames_array shape: (53, 10, 224, 224, 1)
frames_array shape: (85, 10, 224, 224, 1)
frames_array shape: (85, 10, 224, 224, 1)
frames_array shape: (64, 10, 224, 224, 1)
frames_array shape: (64, 10, 224, 224, 1)
frames_array shape: (112, 10, 224, 224, 1)
frames_array shape: (113, 10, 224, 224, 1)
frames_array shape: (13, 10, 224, 224, 1)
frames_array shape: (13, 10, 2

In [4]:
# After all subjects have been processed, write the aggregated index information to a JSON file
index_info_file = os.path.join(destination_dir, 'index_info.json')
with open(index_info_file, 'w') as f:
    json.dump(all_subjects_index_info, f, indent=4)